# Inf2 - Foundations of Data Science
## Semester 2 Week 06: Getting data and web scraping

**Learning outcomes:** In this lab you will learn about getting data, mainly via web scraping. By the end of this lab you should be able to:

- Identify when you need to use web scraping to retrieve data
- Determine whether a proposed web scraping exercise is legal
- Use the Python `requests` module to retrieve a web page in a responsible fashion
- Use the Python `BeautifulSoup` package to isolate and retrieve data from a HTML web page
- Understand how web scraping can be automated

This lab is somewhat inspired by this one 
https://ourcodingclub.github.io/tutorials/webscraping/

This lab is licensed under a [Creative Commons Attribution-ShareAlike 4.0 International License](https://creativecommons.org/licenses/by-sa/4.0/).

You might wish to review the contents of the S1 week 4 lecture before or after this lab.

In [1]:
import re
import pandas as pd
import requests

## A. When do we need to do web scraping to retrieve data?

Web scraping is the process of automating the process of extracting data from a web page.

Spring is in the air (in Edinburgh at least), so we might be inspired to find out more about the birds around us. We'll imagine you want to collect information on the characteristics of UK birds, and put them in a table like this:

| Species            | Length   | Wingspan    | Weight                         | UK passage   | Europe          | UK breeding   |
|:-------------------|:---------|:------------|:-------------------------------|:-------------|:----------------|:--------------|
| Aquatic warbler    | 13cm     | 16.5-19.5cm | 10-14g                         | 42 birds     | 12-20,000 pairs | nan           |
| White-tailed eagle | 70-90cm  | 200-240cm   | 3.5-5kg (male); 4-7kg (female) | nan          | nan             | 106 pairs     |


It’s easy enough to head to the [Royal Society for the Protection of Birds (RSPB) A-Z listing of birds](https://www.rspb.org.uk/birds-and-wildlife/wildlife-guides/bird-a-z/), click through each page, then copy the relevant information and paste it into a spreadsheet. Now imagine you want to repeat this for all 238 birds! This can quickly become very tedious as you click between lots of pages, repeating the same actions over and over again. It also increases the chance of making mistakes when copying and pasting. By automating this process, i.e. web scraping, you can reduce the chance of making mistakes and speed up your data collection. Additionally, once you have written the script, it can be adapted for a lot of different projects, saving time in the long run.

The first rule of web scraping is **don't do it unless you have to**. Data is increasingly available in structured formats such as CSV and JSON. Web scraping seems cool, but it's also time-consuming and fiddly. It pays to spend some time searching for files in structured formats, in which the data may well be in a cleaner format than on a web page. However, if you can't find the data you need in a structured format, but it is available on a web page, then web scraping can be helpful.

## B. Before we start - the law and ethics of web-scraping

It is worth remembering that web scraping involves a relationship between you, the web scraper, and the owner of the website, who may have invested considerable time and money in creating their site. It's possible that by web scraping you could have an adverse effect on either their intellectual property or their web server, so we need to think a little about law and ethics.

The law around web scraping is not always clear ([Byrce Davies - _Is web scraping legal?_](https://scrapediary.com/is-web-scraping-legal/)) but you should always check the terms and conditions of the website before before starting scraping. For example, [the Copyright Policy of the Financial Times website](https://help.ft.com/legal-privacy/copyright-policy/) says you cannot
> Frame, harvest or scrape FT content or otherwise access FT content for similar purposes.

Beyond the legal restrictions for a specific website, we should also consider general ethical principles of web scraping. James Densmore in his article [*Ethics in webscraping*](https://towardsdatascience.com/ethics-in-web-scraping-b96b18136f01) suggests the following set rules for those undertaking web scraping:

> - If you have a public API that provides the data I’m looking for, I’ll use it and avoid scraping all together.
> - I will always provide a User Agent<sup>*</sup> string that makes my intentions clear and provides a way for you to contact me with questions or concerns.
> - I will request data at a reasonable rate. I will strive to never be confused for a [DDoS](https://en.wikipedia.org/wiki/Denial-of-service_attack) attack.
> - I will only save the data I absolutely need from your page. [...]
> - I will respect any content I do keep. I’ll never pass it off as my own.
> - I will look for ways to return value to you. Maybe I can drive some (real) traffic to your site or credit you in an article or post.
> - I will respond in a timely fashion to your outreach and work with you towards a resolution.
> - I will scrape for the purpose of creating new value from the data, not to duplicate it.

<sup>*</sup> We'll explain this later.

Equipped with this information, we'll now start scraping.

### B.1 First check the copyright

**Excercise 01** Take a look at the [Copyright information](https://www.rspb.org.uk/help/copyright) and [Terms and Conditions](https://www.rspb.org.uk/help/terms--conditions/). Can you see anything that would prevent us scraping the information? (Note that education is classed as a non-commercial activity.) What restrictions are there on how you can use the information?

Your answer:

There is nothing that specifically prevents web scraping. We are allowed to use "reasonable extracts" of the website for our our personal use or private study:

> You are permitted to view the contents of this site on screen. You are also permitted to print one copy of reasonable extracts from the site for your personal use or private study and to save one copy of such reasonable extracts to your local hard disk for your personal use or private study.

We are not allowed to use the data for commercial use:

> It is strictly forbidden to copy any material from this website for commercial use (other than as necessary for the purpose of viewing the site in the course of business).

Education isn't a commercial use, so we interpret that we are OK for this exercise. However you couldn't scrape the information from website, and then repost it on your own website.

### B.2 Requesting robots.txt

A web server is just another computer on the internet that is listening for **requests** from other computers on the internet. When we're browsing the web, our web browsers are making requests to web servers all around the internet. The web browser is one example of a **user agent**. We're now going to use the python `requests` package as a user agent.

The first file we will retrieve is called `robots.txt`.

Every web server should have this file to tell robots (i.e. automated user agents like search engine crawlers that move from page to page on the web) what the website owner is happy for them to request. (Note that the file is purely advisory. Bad bots - malware etc - will ignore this file.)

In [2]:
url = 'https://www.rspb.org.uk/robots.txt'
r = requests.get(url)
print(r.text)

User-agent: *
Disallow: /search/
Disallow: /community/SiteMap.ashx
Disallow: /includes/xml/
Sitemap: https://www.rspb.org.uk/sitemapindex.xml


`r` is an request object. `r.text` contains the contents of the `robots.txt` file.

The first line of the file `User-agent: *` tells us that the next section of the file applies to *all* user agents - that includes us! The next lines tell us the areas of the site that should not be scraped automatically. The RSPB don't want us scraping any pages whose URLs start https://www.rspb.org.uk/search ,
https://www.rspb.org.uk/community/SiteMap.ashx etc. There is more information on the format of the `robots.txt` file [here](https://www.robotstxt.org/robotstxt.html).

Note that the web server will have a record of the "User-agent" in its logs. For example, the request we've just made will be recorded in the web server logs like this:
```
77.99.216.20 - - [20/Feb/2021:18:24:40 +0000] "GET /robots.txt HTTP/1.1" 200 4594 "-" "python-requests/2.24.0"
```
This means that a user agent called "python-requests/2.24.0" has made a request from the IP address 77.99.216.20 for the file `robots.txt`. Following the suggested guidelines on ethical web scraping, it would be polite to advertise who we are, by changing the name of the user agent in the request to something like this:

In [3]:
headers = {'user-agent': 'Foundations of Data Science Course (https://www.github.com/Inf2-FDS/weekS2-05-web-scraping)'}
r = requests.get(url, headers=headers)

This request will appear in the server logs like this:
```
77.99.216.20 - - [20/Feb/2021:19:57:16 +0000] "GET /robots.txt HTTP/1.1" 200 4594 "-" "Foundations of Data Science Course (https://www.github.com/Inf2-FDS/weekS2-05-web-scraping)"
```
Uf the site owner gets annoyed by frequent requests, they can at least let us know about the problem. But please, when you web scrape, "request data at a reasonable rate" (as per the ethical guidelines above). I.e. don't make lots of requests in quick succession (see later on how to do this).

Note that the server log also includes the Status code 200 - this means that the request has been delivered successfully by the server. At out end we can check that the code is 200:

In [4]:
r.status_code

200

If the code starts with 2, it means the request was successful. If it starts with 3, it means that the request was successful. If it starts with 4, it means that the page doesn't exist, and if it starts with 5, it indicates an error at the server end.

**Exercise 02:** Make a request to the non-existent page https://www.rspb.org.uk/dodo and report on the error code

In [5]:
r = requests.get("https://www.rspb.org.uk/dodo", headers=headers)
r.status_code

404

## C. Getting our first bird

**Exercise 03:** Get the RSPB web page for the White-tailed eagle:
https://www.rspb.org.uk/birds-and-wildlife/wildlife-guides/bird-a-z/white-tailed-eagle/ and print the text of the page that you have retrieved.

In [6]:
# Your answer
url = 'https://www.rspb.org.uk/birds-and-wildlife/wildlife-guides/bird-a-z/white-tailed-eagle/'
r = requests.get(url, headers=headers)
r.text

'\r\n\r\n    <!DOCTYPE html>\r\n    <!--[if lt IE 7]><html class="no-js lt-ie9 lt-ie8 lt-ie7" lang="en-GB"> <![endif]-->\r\n    <!--[if IE 7]><html class="no-js lt-ie9 lt-ie8" lang="en-GB"> <![endif]-->\r\n    <!--[if IE 8]><html class="no-js lt-ie10 lt-ie9" lang="en-GB"> <![endif]-->\r\n    <!--[if IE 9]><html class="no-js lt-ie10" lang="en-GB"> <![endif]-->\r\n    <!--[if gt IE 9]><!-->\r\n    <html class="no-js" lang=en-GB>\r\n    <!--<![endif]-->\r\n    <head>\r\n        <script>(function (H) { H.className = H.className.replace(/\\bno-js\\b/, \'js\') })(document.documentElement)</script>\r\n\r\n        <meta charset="utf-8" />\r\n        <meta http-equiv="X-UA-Compatible" content="IE=EDGE" />\r\n        <title>White-Tailed Eagle Facts | Haliaeetus Albicilla - The RSPB</title>\r\n        <meta name="description" content="The white tailed eagle is the largest UK bird of prey. It went extinct in the UK during the early 20th century due to illegal killing. Find out more">\r\n<meta prop

### C.1 Quick review of HTML and some useful terminology

What you should see is the Hypertext Markup Language (HTML) source of the page. It's exactly what you would see if you looked at the page source in the browser.

You probably know already, but an HTML document has a nested structure, like this:
```
<!DOCTYPE html>
<html>
  <head>
    <title>Page title</title>
    <script src='https://www.googletagmanager.com/gtag/js?id=UA-11409245-4'/>
  </head>
  <body>
    <h1>Aquatic warbler</h1>
    <p style='color: red'>Stuff about the aquatic warbler</p>
  </body>
</html>
```
The angle brackets containing text are called [tags](https://developer.mozilla.org/en-US/docs/Glossary/Tag). Tags either:
- come in pairs, with an opening tag and a closing tag, e.g. `<h1>...</h1>`
- or they may be a single tag, in which case they have  slash to indicate that they are a closing tag: e.g. `<script ... />`

Some tags have [attributes](https://developer.mozilla.org/en-US/docs/Glossary/Attribute), such as the `style` attribute in the opening `<p>` tag and the `src` attribute in the `<script>` tag.

*Everything* between a pair of tags is an [element](https://developer.mozilla.org/en-US/docs/Glossary/Element). For example the `<html>` element is all the text (apart from the `DOCTYPE`). The `<body>` element contains:
```
<body>
  <h1>Aquatic warbler</h1>
  <p style='color: red'>Stuff about the aquatic warbler</p>
</body>
```

The indentation structure in the code indicates that there is a family tree-like structure to the document. We call the `<html>` the root element. It has two children (`<head>` and `<body`>). In turn `<body>` has two children: `<h1>` and `<p>`. Because `<h1>` and `<p>` have the same parent, they are [siblings](https://dictionary.cambridge.org/us/dictionary/english/sibling).

### C.2 Making BeautifulSoup from the bird page

You can just see some tags in the text returned by our request, but the tree structure is not clear - we just have a string of text. Fortunately there is a Python package called [BeautifulSoup](https://www.crummy.com/software/BeautifulSoup/)<sup>*</sup> that will help use to _parse_ the text so that we can extract the tags and their contents (i.e. the elements) easily.

<sup>*</sup>**Note:** Crazy name, crazy package. The name relates to the a part of the of the plot in an equally crazy book, [*Alice in Wonderland*](http://www.gutenberg.org/ebooks/11) by the mathematician Lewis Carroll, which features a number of birds, including an Eaglet (a baby eagle). Take a minute to look at the video of the White tailed eaglet being fed by its parents on the page we are scraping: https://www.rspb.org.uk/birds-and-wildlife/wildlife-guides/bird-a-z/white-tailed-eagle/

In [7]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(r.content, 'html.parser')
soup


<!DOCTYPE html>

<!--[if lt IE 7]><html class="no-js lt-ie9 lt-ie8 lt-ie7" lang="en-GB"> <![endif]-->
<!--[if IE 7]><html class="no-js lt-ie9 lt-ie8" lang="en-GB"> <![endif]-->
<!--[if IE 8]><html class="no-js lt-ie10 lt-ie9" lang="en-GB"> <![endif]-->
<!--[if IE 9]><html class="no-js lt-ie10" lang="en-GB"> <![endif]-->
<!--[if gt IE 9]><!-->
<html class="no-js" lang="en-GB">
<!--<![endif]-->
<head>
<script>(function (H) { H.className = H.className.replace(/\bno-js\b/, 'js') })(document.documentElement)</script>
<meta charset="utf-8"/>
<meta content="IE=EDGE" http-equiv="X-UA-Compatible"/>
<title>White-Tailed Eagle Facts | Haliaeetus Albicilla - The RSPB</title>
<meta content="The white tailed eagle is the largest UK bird of prey. It went extinct in the UK during the early 20th century due to illegal killing. Find out more" name="description"/>
<meta content="White-Tailed Eagle Facts | Haliaeetus Albicilla - The RSPB" property="og:title">
<meta content="The white tailed eagle is the l

We can now see that HTML code is printed in a somewhat prettier way. In fact BeatifulSoup has reconstructed the parse tree of the document.

### C.3 Accessing tags and elements

We can access the first tag of a particular type using the following syntax:

In [8]:
## The title element
print(soup.title)

## The name of the tag
print(soup.title.name)

## The contents of the tag
print(soup.title.string)

## An img (Image) tag
print(soup.img)

<title>White-Tailed Eagle Facts | Haliaeetus Albicilla - The RSPB</title>
title
White-Tailed Eagle Facts | Haliaeetus Albicilla - The RSPB
<img alt="RSPB - giving nature a home" class="lazyload" data-expand="500" data-src="/static/images/rspb-logo-white.png"/>


Note the last tag we have retrieved is an anchor (i.e. link). It has an attribute `href`. We can extract the contents of the href attribute like this:

In [9]:
soup.img['alt']

'RSPB - giving nature a home'

**Exercise 04:** Get the contents of the first h1 tag:

In [10]:
## Your answer
soup.h1.string

'White-tailed eagle'

Hopefully you've now got the name of the bird, with one line of code.

### C.4 Navigating deeper into the tree

Now to find the length, the wingspan, the weight and the number of UK breeding pairs of the White tailed eagle.

We can navigate down the tree structure like this:

In [11]:
## Find the first `div` element that is the child of the `body` element
soup.body.div

<div class="outer-wrapper" data-behavior="navigation gtm-data block-background">
<div class="inner-wrapper" data-binding="inner-wrapper">
<input name="__RequestVerificationToken" type="hidden" value="ENsKB4q73GLxjVZvl_xeEc82pQ9IwQnb_qNVogklNg_Ry_jHUQJNPWpI9YHDkPKWSslSeGoPA5jPJwKOC1Bs7OjHawbtRVKhYIyxrZxHSak1"/>
<header class="site-header" style="background-image: url(/globalassets/development-images/header--footer/header-bg-new-grid.jpg)">
<div class="container container-fix-tablet">
<div class="row">
<div class="col-12 col-md-12 col-lg-12">
<div class="inner container-fix">
<a class="site-header__logo" href="/" role="banner">
<img alt="RSPB - giving nature a home" class="lazyload" data-expand="500" data-src="/static/images/rspb-logo-white.png"/>
</a>
<div class="site-header__search-form" data-behavior="module-toggle" role="search">
<div class="desktop">
<form action="/link/92a160a2759045049e25ca11bb59d280.aspx">
<a class="link toggle open-search" data-toggle-fade="" href="#">
<span cla

**Exercise 05:** Have a go at navigating the tree by extending the syntax above, e.g.:
```
soup.body.div.div.div.div.div
```

In [12]:
# Your answer
soup.body.div.div.div.div.div.div.div.div.div.div.div.div.div

<div class="inner-content">
<div class="search-field">
<input id="search-main-nav" name="q" placeholder="Enter search term" type="text"/>
<button type="submit">
<!--[if gte IE 9]><!-->
<svg enable-background="new 0 0 595.28 587.447" height="23px" version="1.1" viewbox="0 0 595.28 587.447" width="23px" x="0px" xml:space="preserve" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink" y="0px">
<path d="M594.979,533.824l-72.301-72.301l-57.841-57.841c29.523-41.573,47.598-92.787,47.598-147.615
    C512.435,115.079,397.356,0,256.368,0S0.301,115.079,0.301,256.067s115.079,256.067,256.067,256.067
    c59.046,0,113.272-20.486,156.653-53.624l99.414,99.414c1.205,1.205,2.41,2.41,4.218,3.013l26.511,26.511L594.979,533.824z
     M255.765,451.28c-107.849,0-195.816-87.966-195.816-195.816S147.916,59.648,255.765,59.648s195.816,87.966,195.816,195.816
    c0,45.188-15.063,86.159-40.971,119.297c-8.435,10.845-18.075,21.088-28.318,30.126C347.949,433.807,303.966,451.28,255.765,451.28z
  

You can see that it's going to be tricky navigating a tree like this.

### C.5 The `find_all()` method

Fortunately BeautifulSoup offers other methods of navigating the tree. More often than not the tag we want won't be the first occurrence in the document. If we want to extract all the `<dl>` ("description list") tags we can use the `find_all` function like this:

In [13]:
soup.body.find_all('dl')

[<dl class="species-measurements-population__details">
 <dt class="species-measurements-population__details-label">Length:</dt>
 <dd class="species-measurements-population__details-content">
                                                             70-90cm
                                                         </dd>
 </dl>, <dl class="species-measurements-population__details">
 <dt class="species-measurements-population__details-label">Wingspan:</dt>
 <dd class="species-measurements-population__details-content">
                                                             200-240cm
                                                         </dd>
 </dl>, <dl class="species-measurements-population__details">
 <dt class="species-measurements-population__details-label">Weight:</dt>
 <dd class="species-measurements-population__details-content">
                                                             3.5-5kg (male); 4-7kg (female)
                                                     

This looks more promising. We now have a list of elements that we can iterate over, like this:

In [14]:
measurements = soup.body.find_all('dl')
for measurement in measurements:
    print(measurement.dt.string)
    print(measurement.dd.string)

Length:

                                                            70-90cm
                                                        
Wingspan:

                                                            200-240cm
                                                        
Weight:

                                                            3.5-5kg (male); 4-7kg (female)
                                                        
UK breeding:

                                                            106 pairs
                                                        


However, there is a lot of white space in what we have scraped. How do we get rid of it?

### C.5 Enter regular expressions

You may have encountered [regular expressions](https://en.wikipedia.org/wiki/Regular_expression) before, which are also referred to as **regexs** (or "regexps" if you are a LISP programmer). These are sets of character combinations that python (and many other programming languages) can understand and are used to search through character strings. We don't have space to go into them here in depth, but we will guide you through how to use the python `re` package to remove whitespace.

The `re.sub` function replaces the parts of a string matching a pattern with a desired replacement. E.g.:

In [15]:
re.sub("^\\s+|\\s+$", '', measurements[0].dd.string)

'70-90cm'

 Let’s break down "^\\s+|\\s+$" to understand what it means:

`^` = From start of string

`\\s` = White space

`+` = Select 1 or more instances of the character before

`|` = “or”

`$` = To the end of the line

So `^\\s+|\\s+$` can be interpreted as "Select one or more white spaces that exist at the start of the string and select one or more white spaces that exist at the end of the string".

## D. Putting it all together

We now know everything we need to know to extract the data from the page and create a pandas DataFrame containing the following information:

- Species of bird
- All measurements associated with the bird

### D.1 Text to DataFrame

**Exercise 06:** Create a function that takes the text of a bird species page on the RSPB website (i.e. `r.text`, and returns a Pandas DataFrame called `eagle` with the above elements. Make sure that any whitespace has been removed.

In [16]:
## You answer

def get_measurements(text):
    soup = BeautifulSoup(text)
    out = pd.DataFrame({'Species': [soup.h1.string]})
    measurements = soup.body.find_all('dl')
    for measurement in measurements:
        label = re.sub("^\\s+|\\s+$", '', measurement.dt.string)
        label = re.sub(':', '', label)
        content = re.sub("^\\s+|\\s+$", '', measurement.dd.string)
        out.loc[0, label] = content
    return(out)

eagle = get_measurements(r.text)
eagle

,Species,Length,Wingspan,Weight,UK breeding
0,White-tailed eagle,70-90cm,200-240cm,3.5-5kg (male); 4-7kg (female),106 pairs


### D.2 Joining DataFrames

**Exercise 07:** Use your function to get the corresponding DataFrame called `warbler` for the Aquatic warbler: https://www.rspb.org.uk/birds-and-wildlife/wildlife-guides/bird-a-z/aquatic-warbler/

In [17]:
## Your answer

url = 'https://www.rspb.org.uk/birds-and-wildlife/wildlife-guides/bird-a-z/aquatic-warbler/'
r2 = requests.get(url, headers=headers)
warbler = get_measurements(r2.text)
warbler

,Species,Length,Wingspan,Weight,UK passage,Europe
0,Aquatic warbler,13cm,16.5-19.5cm,10-14g,42 birds,"12-20,000 pairs"


We notice that the set of measurements for the aquatic warbler is not the same as for the eagle - e.g we don't have the `UK breeding` measurement, but we do have a `Europe` measurement.

**Exercise 08:** Create a unified table from the `eagle` and `warbler` DataFrames. If a measurement doesn't exist for one bird, but does for the other.

In [18]:
pd.merge(warbler, eagle, how='outer')

,Species,Length,Wingspan,Weight,UK passage,Europe,UK breeding
0,Aquatic warbler,13cm,16.5-19.5cm,10-14g,42 birds,"12-20,000 pairs",NaN
1,White-tailed eagle,70-90cm,200-240cm,3.5-5kg (male); 4-7kg (female),NaN,NaN,106 pairs


## D.3 Scraping multiple pages

We've now finished the practical part of this lab, but here is how we would proceed if we wanted to automate the scraping of more birds from the RSPB website. To do this we would:
- get the contents of the A-Z listing: https://www.rspb.org.uk/birds-and-wildlife/wildlife-guides/bird-a-z/   (The robots.txt file says this is OK)
- Find the links to the pages with birds which we will find by searching for `<a>` tags and extracting the `href` attributes.
- Create an empty data frame
- For each page:
  - Request the text of the page
  - Run the function that we've just written
  - Store the series in the data frame
  - Run `time.sleep(10)` - this means that there is a pause of 10 seconds before we make the next request. This will reduce the load on the RSPB website, and prevent them from thinking that we are trying to mount a DDoS attack. Web servers can be configured to detect rapid requests in succession and can then block further requests.
  
It gets more complicated, because this there are only 16 birds listed on each A-Z page, and you need to find a way of iterating through each listing page. There is no need to undertake this exercise. If you have to do web scraping you will find that every website is structured differently, and you will need to find a strategy that works for that site.

It is also worth remembering that sites change their structure quite often, so if you programmed a scraper for a site last year, it may not work this year.

## D.4 How did we find the measurements?

You may be wondering how we found the elements containing the measurements in the webpage. This is probably the trickiest part of web scraping.

One way to do this is to use the "Inspector" function in the browser. In Firefox you can do this by pressing `Ctrl-Alt-I`. Then, as in the picture below:

1. Select the "element picker"
2. Click on the part of the page you are interested in (we'll call this the element of interest)
3. Look at the corresponding code. You can copy it by right-clicking on it and selecting **Copy->Outer HTML**.

![Screenshot of Inspector in Firefox](Screenshot-inspector-annotated.png)

The HTML we have found looks like this:
```
<div class="species-measurements-population__measurements">
    <h3 class="key-info-title-measurements">Measurements:</h3>
    <dl class="species-measurements-population__details">
        <dt class="species-measurements-population__details-label">Length:</dt>
        <dd class="species-measurements-population__details-content">
            13cm
        </dd>
    </dl>

    <dl class="species-measurements-population__details">
        <dt class="species-measurements-population__details-label">Wingspan:</dt>
        <dd class="species-measurements-population__details-content">
            16.5-19.5cm
        </dd>
    </dl>

    <dl class="species-measurements-population__details">
        <dt class="species-measurements-population__details-label">Weight:</dt>
        <dd class="species-measurements-population__details-content">
            10-14g
        </dd>
   </dl>
</div>
```

We now need to work out how to retrieve these elements. We noticed that the elements are all in a `div` with the class `species-measurements-population__measurements`, and this informed the strategy that we've used above.

## The last FDS lab

This was the last FDS lab. We hope you enjoyed it! Please do leave feedback in the form.

![Disappearing cheshire cat. Public domain, downloaded from https://www.alice-in-wonderland.net](alice-in-wonderland-cheshire-cat-vanishing.jpg)
The Cheshire cat disappears in *Alice in Wonderland*.